# Video Game Sales Data Analysis 
---
This Jupyter notebook will explore a dataset of international video game sales in order to identify patterns that determine whether a game succeeds or not.


#### Project Sections 
1. Initial Set Up and Data Preparation 
2. General Data Analysis 
3. Analysis By Sales Region 
4. Hypothesis Testing 

### Initial Set Up & Data Preparation 
---

In [137]:
# Import required libraries 
from scipy import stats as st
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

# Read in Data
df = pd.read_csv('games.csv')

#### Data Cleaning 
Part 1: Finding Issues 

In [138]:
# Sample Data 
df.sample(3)

,Name,Platform,Year_of_Release,Genre,NA_sales,EU_sales,JP_sales,Other_sales,Critic_Score,User_Score,Rating
8724,Chessmaster: The Art of Learning,DS,2007.0,Misc,0.12,0.03,0.0,0.01,NaN,6.7,E
7306,NBA,PSP,2005.0,Sports,0.20,0.00,0.0,0.02,57.0,6.9,E
5987,Area 51,XB,2005.0,Shooter,0.22,0.06,0.0,0.01,72.0,8.1,M


In [139]:
# Check data types and Missing values 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16713 non-null  object 
 1   Platform         16715 non-null  object 
 2   Year_of_Release  16446 non-null  float64
 3   Genre            16713 non-null  object 
 4   NA_sales         16715 non-null  float64
 5   EU_sales         16715 non-null  float64
 6   JP_sales         16715 non-null  float64
 7   Other_sales      16715 non-null  float64
 8   Critic_Score     8137 non-null   float64
 9   User_Score       10014 non-null  object 
 10  Rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB


In [140]:
# Check for duplicate rows 
df.duplicated().sum()

0

In [141]:
# Check for implicit duplicates
df['Platform'].unique()

array(['Wii', 'NES', 'GB', 'DS', 'X360', 'PS3', 'PS2', 'SNES', 'GBA',
       'PS4', '3DS', 'N64', 'PS', 'XB', 'PC', '2600', 'PSP', 'XOne',
       'WiiU', 'GC', 'GEN', 'DC', 'PSV', 'SAT', 'SCD', 'WS', 'NG', 'TG16',
       '3DO', 'GG', 'PCFX'], dtype=object)

In [142]:
df['Genre'].unique()

array(['Sports', 'Platform', 'Racing', 'Role-Playing', 'Puzzle', 'Misc',
       'Shooter', 'Simulation', 'Action', 'Fighting', 'Adventure',
       'Strategy', nan], dtype=object)

In [143]:
df['Rating'].unique()

array(['E', nan, 'M', 'T', 'E10+', 'K-A', 'AO', 'EC', 'RP'], dtype=object)

Part 2: Fixing Issues  
Upon initial exploration of the data, it has been determined that the following tasks must be carried out in order to prepare the data for analysis: 
- Column names should be made lowercase 
- Columns with object data types should have strings in lowercase
- year_of_release should be in datetime data type
- user_score should be a float data type
- Replace missing values: 
    - rating: Replace null with 'unrated'
    - critic_score and user_score: Replace null and 'tbd' with median of genre 
    - genre: replace null with 'misc'
    - name: drop null values (this is 0.01% of data)
    - year_of_release: drop rows with null values (this is 1.6% of data)
- Add a column with total sales 


In [144]:
# Make column names lowercase 
df = df.rename(
    columns={
        'Name':'name',
        'Platform':'platform',
        'Year_of_Release':'year_of_release',
        'Genre':'genre',
        'NA_sales':'na_sales',
        'EU_sales':'eu_sales',
        'JP_sales':'jp_sales',
        'Other_sales':'other_sales',
        'Critic_Score':'critic_score',
        'User_Score':'user_score',
        'Rating':'rating'
    }
)

# Make the contents of object datatype columns lowercase 
df['name'] = df['name'].str.lower()
df['platform'] = df['platform'].str.lower()
df['genre'] = df['genre'].str.lower()
df['rating'] = df['rating'].str.lower()

# Verify Changes
df.sample(1)

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
5366,battalion wars 2,wii,2007.0,strategy,0.22,0.02,0.08,0.02,75.0,7.6,t


In [145]:
# Make year_of_release the date time datatype 
df['year_of_release'] = pd.to_datetime(df['year_of_release'], format='%Y')
df['year_of_release'] = df['year_of_release']

# Make user_score a float datatype and get rid of 'tbd'
df['user_score'] = pd.to_numeric(df['user_score'], errors='coerce')

# Replace null values in ratings column with 'unrated'
df['rating'] = df['rating'].fillna('unrated')

# Replace null values in genre column with 'misc'
df['genre'] = df['genre'].fillna('misc')

# Replace null values in critic_score column with median for genre
grouped_avg = df.groupby('genre')['critic_score'].transform('median')
df['critic_score'] = df['critic_score'].fillna(grouped_avg)

# Replace null values in user_score column with median for genre group
grouped_avg = df.groupby('genre')['user_score'].transform('median')
df['user_score'] = df['user_score'].fillna(grouped_avg)

# Drop rows with null values in the name or year_of_release columns 
df= df.dropna(axis='rows')

In [146]:
# Add a column with total sales 
df['total_sales'] = df['eu_sales'] + df['jp_sales'] + df['na_sales'] + df['other_sales']

In [147]:
# Verify Changes 
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16444 entries, 0 to 16714
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   name             16444 non-null  object        
 1   platform         16444 non-null  object        
 2   year_of_release  16444 non-null  datetime64[ns]
 3   genre            16444 non-null  object        
 4   na_sales         16444 non-null  float64       
 5   eu_sales         16444 non-null  float64       
 6   jp_sales         16444 non-null  float64       
 7   other_sales      16444 non-null  float64       
 8   critic_score     16444 non-null  float64       
 9   user_score       16444 non-null  float64       
 10  rating           16444 non-null  object        
 11  total_sales      16444 non-null  float64       
dtypes: datetime64[ns](1), float64(7), object(4)
memory usage: 1.6+ MB


In [148]:
df.sample(3)

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,total_sales
2357,the tomb raider trilogy,ps3,2011-01-01,action,0.27,0.46,0.00,0.15,78.0,8.1,t,0.88
12478,shonen jump's shaman king: legacy of the spiri...,gba,2005-01-01,action,0.04,0.02,0.00,0.00,68.0,7.4,unrated,0.06
9000,"lunar 2: eternal blue(sales, but wrong system)",gen,1994-01-01,role-playing,0.00,0.00,0.14,0.00,74.0,7.8,unrated,0.14


### General Data Analysis
---
In this section, the entire dataset will be analyzed in order to answer some questions about global sales. The following questions will be answered: 
- How long does it generally take for new platforms to appear and old ones to fade?
- Which platforms are leading in sales? Which are growing and shrinking?
- Are the differences in global sales of games for each platform significant? What about average sales on various platforms? 
- Is there a correlation between user reviews and sales for the X platform?
- How do the sales of BLANK on BLANK compare to sales of the same game on BLANK?
- Which genres are the most profitable? Which are the least?

#### Is the data for every period significant?
Volume of games released per year will determine if every period in the data set is significant. 

#### How long does it generally take for new platforms to appear and old ones to fade?
To answer this question, sales variation from platform to platform will be analyzed to determine the platforms with the greatest total sales. Distributions will be built based on data from each year in order to analyze the rise and fall of these platforms. 

#### Which platforms are leading in sales? Which are growing and shrinking?

#### Are the differences in global sales of games for each significant? What about average sales on various platforms? 

#### Is there a correlation between user reviews and sales for the X platform?

#### How do the sales of BLANK on BLANK compare to sales of the same game on BLANK?

#### Which genres are the most profitable? Which are the least?

### Analysis by Sales Region
---
In this section, comparisons will be made between the three sales regions: North America, Europe, and Japan. The following factors will be analyzed:
- Platform market share variations by region
- Genre popularity variation across regions 
- Effect ESRB rating by region 

#### Platform Market Share Variations Across Regions

#### Genre Popularity Variations Across Regions

#### ESRB Rating Effect by Region 

### Hypothesis Testing 
---
This section will utilize hypothesis testing to see if the following two hypotheses are supported through statistical analysis:
1. Average user ratings of the Xbox One and PC platforms are the same. 
2. Average user ratings for the Action and Sports genres are different.

#### Are the average user ratings of the Xbox One and PC platforms the same?
- Null Hypothesis: There is a statistcally significant difference between the average user ratings of the Xbox One and PC platforms.
- Alternative Hypothesis: There is not a statistcally significant difference between the average user ratings of the Xbox One and PC platforms.

#### Are the average user ratings for the Action and Sports genres are different? 
- Null Hypothesis: There is not a statistcally significant difference between the average user ratings for the Action and Sports genres.
- Alternative Hypothesis: There is a statistcally significant difference between the average user ratings for the Action and Sports genres.



## Conclusions 
---